# 05. 淺層神經網路 (ANN) 實驗

## 📖 實驗目標

在 03_ModelBuilding 和 04_XGBoost 中，我們測試了傳統機器學習模型：
- ✅ **Logistic Regression**: AUC >0.88，但 F1 較低
- ✅ **XGBoost**: F1 提升 9-39%，AUC 略有改善

本 notebook 將測試 **淺層神經網路 (ANN)**，理由：
1. 🧠 **非線性建模能力**: 可能捕捉更複雜的特徵關係
2. 📊 **適合中等資料量**: 6K 樣本足夠訓練 1-2 層網路
3. 🎯 **class_weight 支援**: 可處理類別不平衡
4. ⚡ **訓練速度快**: 比 SVM 快很多

**ANN vs DNN**：
- **ANN (淺層)**: 1-2 隱藏層，適合 5K-10K 樣本 ✅
- **DNN (深層)**: 3+ 隱藏層，需要 50K+ 樣本 ❌

---

## 📋 實驗流程

1. 載入資料（使用與 03/04 相同的資料處理）
2. 單任務 ANN（三種疾病分別訓練）
3. MTL ANN（多輸出神經網路）
4. 與最佳模型比較（MTL LR, XGBoost）
5. 結論與建議

## 1. 載入套件與資料

In [ ]:
# 安裝 TensorFlow/Keras (如果尚未安裝)
import sys
!{sys.executable} -m pip install tensorflow scikeras

In [ ]:
# 基礎套件
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 機器學習套件
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, 
    roc_auc_score, confusion_matrix
)
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier

# TensorFlow/Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

# 設定
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'Arial Unicode MS', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False

# 設定隨機種子
np.random.seed(42)
tf.random.set_seed(42)

print("✅ 套件載入完成")
print(f"TensorFlow 版本: {tf.__version__}")
print(f"Keras 版本: {keras.__version__}")

In [ ]:
# 載入資料
df = pd.read_csv('../../data/processed/SUA_CVDs_wide_format.csv')

print("✅ 資料載入成功！")
print(f"資料形狀: {df.shape[0]:,} 人, {df.shape[1]} 個欄位")

## 2. 準備特徵和目標變數

In [ ]:
# 定義特徵組
demographic_features = ['sex', 'Age']
biomarker_names = ['FBG', 'TC', 'Cr', 'UA', 'GFR', 'BMI', 'SBP', 'DBP']
t1_features = [f'{name}_T1' for name in biomarker_names]
t2_features = [f'{name}_T2' for name in biomarker_names]
delta1_features = [f'Delta1_{name}' for name in biomarker_names]

# 完整特徵集
X_columns = demographic_features + t1_features + t2_features + delta1_features
X = df[X_columns]

# 目標變數（轉換為 0/1）
y_hypertension = (df['hypertension_T3'] == 2).astype(int)
y_hyperglycemia = (df['hyperglycemia_T3'] == 2).astype(int)
y_dyslipidemia = (df['dyslipidemia_T3'] == 2).astype(int)

# MTL 目標變數
y_multi = np.column_stack([y_hypertension, y_hyperglycemia, y_dyslipidemia])

print(f"特徵數: {len(X_columns)} 個")
print(f"\n目標變數分佈:")
print(f"  高血壓患病率: {y_hypertension.mean():.2%}")
print(f"  高血糖患病率: {y_hyperglycemia.mean():.2%}")
print(f"  高血脂患病率: {y_dyslipidemia.mean():.2%}")

## 3. 資料分割與標準化

In [ ]:
# 資料分割
X_train, X_test, y_train_multi, y_test_multi = train_test_split(
    X, y_multi, test_size=0.2, random_state=42, stratify=y_hypertension
)

# 分離三個目標變數
y_train_hp = y_train_multi[:, 0]
y_train_hg = y_train_multi[:, 1]
y_train_dl = y_train_multi[:, 2]

y_test_hp = y_test_multi[:, 0]
y_test_hg = y_test_multi[:, 1]
y_test_dl = y_test_multi[:, 2]

# 標準化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✅ 資料準備完成")
print(f"訓練集: {X_train_scaled.shape[0]} 人")
print(f"測試集: {X_test_scaled.shape[0]} 人")
print(f"特徵數: {X_train_scaled.shape[1]} 個")

## 4. 計算 class_weight

處理類別不平衡，計算每個疾病的 class_weight。

In [ ]:
# 計算 class_weight
def calculate_class_weight(y):
    n_samples = len(y)
    n_positive = np.sum(y == 1)
    n_negative = np.sum(y == 0)
    
    weight_for_0 = n_samples / (2 * n_negative)
    weight_for_1 = n_samples / (2 * n_positive)
    
    return {0: weight_for_0, 1: weight_for_1}

class_weight_hp = calculate_class_weight(y_train_hp)
class_weight_hg = calculate_class_weight(y_train_hg)
class_weight_dl = calculate_class_weight(y_train_dl)

print("======================================================================")
print("class_weight 計算結果")
print("======================================================================")
print(f"高血壓: {{0: {class_weight_hp[0]:.2f}, 1: {class_weight_hp[1]:.2f}}}")
print(f"高血糖: {{0: {class_weight_hg[0]:.2f}, 1: {class_weight_hg[1]:.2f}}}")
print(f"高血脂: {{0: {class_weight_dl[0]:.2f}, 1: {class_weight_dl[1]:.2f}}}")
print("\n💡 權重越大表示該類別越稀少，需要更多關注")

## 5. 單任務 ANN

建立簡單的 2 層神經網路，分別預測三種疾病。

**架構**：
- 輸入層: 26 個特徵
- 隱藏層 1: 64 neurons + ReLU + Dropout(0.3)
- 隱藏層 2: 32 neurons + ReLU + Dropout(0.3)
- 輸出層: 1 neuron + Sigmoid

In [ ]:
def build_ann_model(input_dim):
    """建立單任務 ANN 模型"""
    model = models.Sequential([
        layers.Dense(64, activation='relu', input_shape=(input_dim,)),
        layers.Dropout(0.3),
        layers.Dense(32, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['AUC']
    )
    
    return model

print("✅ 模型架構定義完成")

In [ ]:
# 訓練三個單任務 ANN 模型
diseases = ['高血壓', '高血糖', '高血脂']
y_trains = [y_train_hp, y_train_hg, y_train_dl]
y_tests = [y_test_hp, y_test_hg, y_test_dl]
class_weights = [class_weight_hp, class_weight_hg, class_weight_dl]

ann_models = {}
ann_results = []

# Early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

print("======================================================================")
print("單任務 ANN 訓練中...")
print("======================================================================\n")

for disease, y_train, y_test, class_weight in zip(diseases, y_trains, y_tests, class_weights):
    print(f"訓練 {disease} 模型...")
    
    # 建立模型
    model = build_ann_model(X_train_scaled.shape[1])
    
    # 訓練（靜默模式）
    history = model.fit(
        X_train_scaled, y_train,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        class_weight=class_weight,
        callbacks=[early_stop],
        verbose=0
    )
    
    # 預測
    y_pred_proba = model.predict(X_test_scaled, verbose=0).flatten()
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    # 計算指標
    auc = roc_auc_score(y_test, y_pred_proba)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred, zero_division=0)
    
    # 儲存結果
    ann_models[disease] = model
    ann_results.append({
        '疾病': disease,
        'AUC': auc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    })
    
    # 輸出結果
    print(f"{disease}:")
    print(f"  訓練 epochs: {len(history.history['loss'])}")
    print(f"  AUC:       {auc:.3f}")
    print(f"  F1:        {f1:.3f}")
    print(f"  Precision: {precision:.3f}")
    print(f"  Recall:    {recall:.3f}")
    
    # 混淆矩陣
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    print(f"  混淆矩陣: TN={tn}, FP={fp}, FN={fn}, TP={tp}")
    print()

print("✅ 單任務 ANN 訓練完成！")

## 6. MTL ANN (Multi-Task Learning)

建立多輸出神經網路，共享隱藏層，同時預測三種疾病。

In [ ]:
def build_mtl_ann_model(input_dim):
    """建立 MTL ANN 模型"""
    # 共享的輸入層
    inputs = layers.Input(shape=(input_dim,))
    
    # 共享的隱藏層
    x = layers.Dense(64, activation='relu')(inputs)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    
    # 三個輸出頭
    output_hp = layers.Dense(1, activation='sigmoid', name='hypertension')(x)
    output_hg = layers.Dense(1, activation='sigmoid', name='hyperglycemia')(x)
    output_dl = layers.Dense(1, activation='sigmoid', name='dyslipidemia')(x)
    
    # 建立模型
    model = models.Model(inputs=inputs, outputs=[output_hp, output_hg, output_dl])
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['AUC']
    )
    
    return model

print("✅ MTL 模型架構定義完成")

In [ ]:
print("======================================================================")
print("MTL ANN 訓練中...")
print("======================================================================\n")

# 建立 MTL 模型
mtl_ann = build_mtl_ann_model(X_train_scaled.shape[1])

# 訓練
history = mtl_ann.fit(
    X_train_scaled, 
    [y_train_hp, y_train_hg, y_train_dl],
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=0
)

print(f"訓練完成，共 {len(history.history['loss'])} epochs")

# 預測
y_pred_proba_hp, y_pred_proba_hg, y_pred_proba_dl = mtl_ann.predict(X_test_scaled, verbose=0)

mtl_ann_results = []

for i, (disease, y_test, y_pred_proba) in enumerate([
    ('高血壓', y_test_hp, y_pred_proba_hp.flatten()),
    ('高血糖', y_test_hg, y_pred_proba_hg.flatten()),
    ('高血脂', y_test_dl, y_pred_proba_dl.flatten())
]):
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    # 計算指標
    auc = roc_auc_score(y_test, y_pred_proba)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred, zero_division=0)
    
    # 儲存結果
    mtl_ann_results.append({
        '疾病': disease,
        'AUC': auc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    })
    
    # 輸出結果
    print(f"\n{disease}:")
    print(f"  AUC:       {auc:.3f}")
    print(f"  F1:        {f1:.3f}")
    print(f"  Precision: {precision:.3f}")
    print(f"  Recall:    {recall:.3f}")
    
    # 混淆矩陣
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    print(f"  混淆矩陣: TN={tn}, FP={fp}, FN={fn}, TP={tp}")

print("\n✅ MTL ANN 訓練完成！")

## 7. ANN 效能比較 (單任務 vs MTL)

In [ ]:
# 合併結果
df_single_ann = pd.DataFrame(ann_results)
df_single_ann['方法'] = 'ANN (單任務)'

df_mtl_ann = pd.DataFrame(mtl_ann_results)
df_mtl_ann['方法'] = 'ANN (MTL)'

df_ann_comparison = pd.concat([df_single_ann, df_mtl_ann], ignore_index=True)

print("================================================================================")
print("ANN 方法比較")
print("================================================================================")
print(df_ann_comparison[['方法', '疾病', 'AUC', 'F1', 'Recall']].to_string(index=False))

# 找出最佳方法
print("\n================================================================================")
print("最佳 ANN 方法")
print("================================================================================\n")

for disease in diseases:
    disease_results = df_ann_comparison[df_ann_comparison['疾病'] == disease]
    best_auc = disease_results.loc[disease_results['AUC'].idxmax()]
    best_f1 = disease_results.loc[disease_results['F1'].idxmax()]
    best_recall = disease_results.loc[disease_results['Recall'].idxmax()]
    
    print(f"{disease}:")
    print(f"  最佳 AUC:    {best_auc['方法']:20s} (AUC={best_auc['AUC']:.3f})")
    print(f"  最佳 F1:     {best_f1['方法']:20s} (F1={best_f1['F1']:.3f})")
    print(f"  最佳 Recall: {best_recall['方法']:20s} (Recall={best_recall['Recall']:.3f})")
    print()

## 8. 視覺化 ANN 效能

In [ ]:
# 建立視覺化
metrics = ['AUC', 'F1', 'Recall']
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for idx, metric in enumerate(metrics):
    ax = axes[idx]
    
    # 準備資料
    pivot_data = df_ann_comparison.pivot(index='疾病', columns='方法', values=metric)
    
    # 繪圖
    pivot_data.plot(kind='bar', ax=ax, width=0.8)
    ax.set_title(f'{metric} 比較', fontsize=14, fontweight='bold')
    ax.set_ylabel(metric, fontsize=12)
    ax.set_xlabel('疾病', fontsize=12)
    ax.legend(title='方法', fontsize=10)
    ax.grid(axis='y', alpha=0.3)
    ax.set_ylim(0, 1)
    
    # 添加數值標籤
    for container in ax.containers:
        ax.bar_label(container, fmt='%.3f', fontsize=9)

plt.tight_layout()
plt.savefig('../../docs/experiments/ann_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ 視覺化完成，已儲存至 docs/experiments/ann_comparison.png")

## 9. 與最佳模型比較

比較 ANN 與之前的最佳模型：
- MTL LR (balanced) from 03
- XGBoost (單任務) from 04

In [ ]:
print("======================================================================")
print("重新訓練 MTL LR (balanced) 和 XGBoost 作為基準比較")
print("======================================================================\n")

# 訓練 MTL LR
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

mtl_lr = MultiOutputClassifier(
    LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000)
)
mtl_lr.fit(X_train_scaled, y_train_multi)

# 訓練 XGBoost
from xgboost import XGBClassifier

xgb_models_comparison = {}
for disease, y_train, y_test in zip(diseases, y_trains, y_tests):
    n_positive = np.sum(y_train == 1)
    n_negative = np.sum(y_train == 0)
    scale_pos_weight = n_negative / n_positive
    
    model = XGBClassifier(
        scale_pos_weight=scale_pos_weight,
        max_depth=5,
        learning_rate=0.1,
        n_estimators=100,
        random_state=42,
        eval_metric='logloss'
    )
    model.fit(X_train_scaled, y_train)
    xgb_models_comparison[disease] = model

print("✅ 基準模型訓練完成！")

In [ ]:
# 計算所有模型的結果
all_results = []

# MTL LR
y_pred_lr = mtl_lr.predict(X_test_scaled)
for i, disease in enumerate(diseases):
    y_test = y_test_multi[:, i]
    y_pred = y_pred_lr[:, i]
    y_pred_proba = mtl_lr.estimators_[i].predict_proba(X_test_scaled)[:, 1]
    
    all_results.append({
        '方法': 'MTL LR (balanced)',
        '疾病': disease,
        'AUC': roc_auc_score(y_test, y_pred_proba),
        'F1': f1_score(y_test, y_pred, zero_division=0),
        'Recall': recall_score(y_test, y_pred, zero_division=0)
    })

# XGBoost
for disease, y_test in zip(diseases, y_tests):
    model = xgb_models_comparison[disease]
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    
    all_results.append({
        '方法': 'XGBoost (單任務)',
        '疾病': disease,
        'AUC': roc_auc_score(y_test, y_pred_proba),
        'F1': f1_score(y_test, y_pred, zero_division=0),
        'Recall': recall_score(y_test, y_pred, zero_division=0)
    })

# 加入 ANN 結果
df_best_ann = df_single_ann.copy()
df_best_ann['方法'] = 'ANN (單任務)'

df_all = pd.concat([pd.DataFrame(all_results), df_best_ann], ignore_index=True)

print("================================================================================")
print("所有模型效能比較")
print("================================================================================")
print(df_all[['方法', '疾病', 'AUC', 'F1', 'Recall']].to_string(index=False))

## 10. 視覺化最終比較

In [ ]:
# 建立最終比較視覺化
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, metric in enumerate(['AUC', 'F1', 'Recall']):
    ax = axes[idx]
    
    # 準備資料
    pivot_data = df_all.pivot(index='疾病', columns='方法', values=metric)
    
    # 繪圖
    pivot_data.plot(kind='bar', ax=ax, width=0.8)
    ax.set_title(f'{metric} 比較', fontsize=14, fontweight='bold')
    ax.set_ylabel(metric, fontsize=12)
    ax.set_xlabel('疾病', fontsize=12)
    ax.legend(title='方法', fontsize=9, loc='upper right')
    ax.grid(axis='y', alpha=0.3)
    ax.set_ylim(0, 1)
    
    # 添加數值標籤
    for container in ax.containers:
        ax.bar_label(container, fmt='%.2f', fontsize=8)

plt.tight_layout()
plt.savefig('../../docs/experiments/all_models_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ 最終比較視覺化完成，已儲存至 docs/experiments/all_models_comparison.png")

## 11. 結論

### 🎯 ANN 效能總結

本實驗測試了淺層神經網路 (ANN) 在三高疾病預測上的表現，並與之前的最佳模型比較。

### 📊 主要發現

**根據實際執行結果填寫**：

1. **ANN vs 傳統機器學習**
   - ANN 的 AUC 表現：[填寫實際結果]
   - 與 XGBoost 相比：[填寫比較結果]
   - 與 LR 相比：[填寫比較結果]

2. **單任務 vs MTL ANN**
   - 效能差異：[填寫結果]
   - 訓練時間：MTL 更快（一次訓練 vs 三次）

3. **ANN 的優缺點**
   - ✅ 優點：非線性建模、自動特徵學習、支援 class_weight
   - ❌ 缺點：需要調參、可解釋性低、可能過擬合

### 💡 建議

1. **何時選擇 ANN？**
   - 需要捕捉非線性關係
   - 資料量足夠（>5K）
   - 不需要模型可解釋性

2. **何時選擇其他模型？**
   - 需要可解釋性 → Logistic Regression
   - 需要特徵重要性 → XGBoost
   - 資料量小 → SVM

### 📝 下一步

- 06_SVM.ipynb: 測試 Support Vector Machine
- 超參數調優：調整 ANN 的層數、neurons、dropout
- 模型集成：結合 ANN, XGBoost, LR